<a href="https://www.kaggle.com/code/yibinx/telco-churn-tactics?scriptVersionId=122939693" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Telecom Customer Churn

In [ ]:
df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
df

## Checking for null values

In [ ]:
df.info()

## EDA

### Graph EDA

In [ ]:
def categorical_vs_churn(column):
    table = df.groupby([column])['Churn'].value_counts().unstack().divide(df.groupby([column])['Churn'].value_counts().unstack().sum(axis=1),axis=0)*100
    table.plot(kind="bar", stacked=True, title=column + ' vs churn',ylabel="Percent churn")
    return table

### Gender vs Churn

In [ ]:
categorical_vs_churn('gender')

The chart does not tell that there is a difference between whether females or males churn more.

### Senior Citizen vs churn

In [ ]:
categorical_vs_churn('SeniorCitizen')

The graph shows that senior citizens are more likely to churn.

### Partner vs Churn

In [ ]:
categorical_vs_churn('Partner')

People with a partner are less likely to churn.

### Dependents vs Churn

In [ ]:
categorical_vs_churn('Dependents')

People with dependents are less likely to churn.

### Tenure vs Churn

In this section, I will calculate the probability that a customer will churn at each tenure level. I will use the formula 
$$ P(churn | tenure = x) = \frac{P(churn \cap tenure = x)}{P(tenure)} $$
for the values x = 0 to 72 months. I will then plot the graph for the correlation between tenure and the probability of churn.

In [ ]:
tenure_churn = []
for i in df.tenure.unique():
    tenure_churn.append([i,len(df[(df.tenure == i) & (df.Churn == 'Yes')])/len(df[df.tenure == i])])
tenure_churn_table = pd.DataFrame(tenure_churn, columns=['Tenure', 'Churn probability'])

In [ ]:
tenure_churn_table.sort_values(by=['Tenure']).plot(x='Tenure',y='Churn probability',title='Tenure vs Churn',ylabel="Churn probability")

As the customer stays with the provider for longer, the probability that it will churn will decrease. However, the probability that it will churn will go back up and then down.

### Phone service vs churn

In [ ]:
categorical_vs_churn('PhoneService')

I cannot tell whether there is a difference in churn rate between those with phone service and those without phone service.

### Multiple lines vs churn

In [ ]:
categorical_vs_churn('MultipleLines')

Among those with phone service, those with multiple lines are more likely to churn than those with just one line.

### Internet Service vs Churn

In [ ]:
categorical_vs_churn('InternetService')

People with fiber optic are most likely to churn. People with DSL internet are more likely to churn than those with no internet.

### Online Security vs churn

In [ ]:
categorical_vs_churn('OnlineSecurity')

Those without online security are more likely to churn.

### Online Backup vs Churn

In [ ]:
categorical_vs_churn('OnlineBackup')

People without online backup are more likely to churn.

### Device protection vs Churn

In [ ]:
categorical_vs_churn('DeviceProtection')

People without device protection are more likely to churn.

### Tech Support vs Churn

In [ ]:
categorical_vs_churn('TechSupport')

People without tech support are more likely to churn.

### Streaming TV vs churn

In [ ]:
categorical_vs_churn('StreamingTV')

The graph does not show that there is a significant difference between the churn rate of those with streaming TV and those without streaming TV.

### Streaming Movies vs churn

In [ ]:
categorical_vs_churn('StreamingMovies')

The graph does not show a significant difference between the churn rate for those with streaming movies and that for those without streaming movies.

### Contract vs Churn

In [ ]:
categorical_vs_churn('Contract')

Those with a two tear contract are least likely to churn. Those with a month-to-month contract are most likely to churn.

### Paperless billing vs churn

In [ ]:
categorical_vs_churn('PaperlessBilling')

People with paperless billing are more likely to churn.

### Payment method vs churn

In [ ]:
categorical_vs_churn('PaymentMethod')

Those with electronic check are most likely to churn. The churn rate for those with bank transfer, electronic check, and mailed check are similar.

### Monthly Charges vs Churn

In [ ]:
mc_churn = []
for i in df.MonthlyCharges.unique():
    mc_churn.append([i,len(df[(df.MonthlyCharges == i) & (df.Churn == 'Yes')])/len(df[df.MonthlyCharges == i])])
mc_table = pd.DataFrame(mc_churn, columns=['Monthly Charges', 'Churn probability'])

In [ ]:
mc_table['Monthly Charges'] = mc_table['Monthly Charges'] // 10

In [ ]:
mc_table['Monthly Charges'] = mc_table['Monthly Charges'] * 10
mc_table.groupby('Monthly Charges').mean().plot()

As the monthly charges increase, the probability to churn increases slightly.

### Total Charges vs Churn

In [ ]:
df.loc[df.tenure == 0, 'TotalCharges'] = 0
df['TotalCharges'] = df['TotalCharges'].astype(float)
tc_churn = []
for i in df.TotalCharges.unique():
    tc_churn.append([i,len(df[(df.TotalCharges == i) & (df.Churn == 'Yes')])/len(df[df.TotalCharges == i])])
tc_table = pd.DataFrame(tc_churn, columns=['Total Charges', 'Churn probability'])

In [ ]:
tc_table

In [ ]:
tc_table['Total Charges'] = tc_table['Total Charges'].astype(float)
tc_table['Total Charges'] = tc_table['Total Charges'] // 1000
tc_table['Total Charges'] = tc_table['Total Charges'] * 1000
tc_table.groupby('Total Charges').mean().plot()

As the total charges increase, the probability of churning decreases. This is likely caused by the increase in tenure.

### Hypothesis testing to determine important variables

In [ ]:
df.columns

In [ ]:
from scipy.stats import chi2_contingency
for i in df.columns:
    c, p, dof, expected = chi2_contingency(pd.crosstab(df[i], df['Churn']))
    if p < .05:
        print(i , p)

## Feature Engineering

### Changing categorical variables to 1's and 0's

In [ ]:
df['Female'] = pd.Series(np.where(df.gender.values == 'Female', 1, 0),df.index)
df = df.drop('gender',axis=1)

### Converting yes and no.

In [ ]:
for i in ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']:
    df[i] = df[i].map({'Yes': 1, 'No': 0})

In [ ]:
df = df.fillna(0)

### One-hot encoding with non-binary categorical columns

In [ ]:
df = df.join(pd.get_dummies(df['InternetService'],prefix='InternetService').drop('InternetService_No',axis=1))
df=df.drop('InternetService',axis=1)

In [ ]:
df = df.join(pd.get_dummies(df['PaymentMethod'],prefix='PaymentMethod'))
df = df.join(pd.get_dummies(df['Contract'],prefix='Contract'))
df=df.drop(['PaymentMethod','Contract'],axis=1)

### Removing multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import seaborn as sns
for i in range(len(np.where(df.corr() > .6)[0])):
    if np.where(df.corr() > .6)[0][i] != np.where(df.corr() > .6)[1][i]:
        print(df.corr().columns[np.where(df.corr() > .6)[1][i]],df.corr().columns[np.where(df.corr() > .6)[0][i]])

In [ ]:
X = df.drop(['customerID','Churn'],axis=1)
vif = [[var,variance_inflation_factor(X.values, X.columns.get_loc(var))] for var in X.columns]
df[np.array(list(filter(lambda x: x[1] < 5, vif)))[:,0]]

## Predicting customer churn

### Logistic Regression

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import seaborn as sns
for i in range(len(np.where(df.corr() > .6)[0])):
    if np.where(df.corr() > .6)[0][i] != np.where(df.corr() > .6)[1][i]:
        print(df.corr().columns[np.where(df.corr() > .6)[1][i]],df.corr().columns[np.where(df.corr() > .6)[0][i]])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
numeric_features = ["tenure", "MonthlyCharges"]
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features)
    ]
)
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)
y = df.Churn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

## Now to implement the tactics

The original questions asked in the beginning of the notebook is how to increase the revenue of a company. To do that, I will have to find out which features lead to churn. For that, I will have to look at which services are causing churn. If a feature has a high feature importance, then it is leading to churn. If a feature has low feature importance, then a lack of it leads to churn. 

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
feature_importance = pd.DataFrame(np.array([X.columns,lr.coef_[0]]).T).sort_values(1)
keep = X.drop(['Dependents','tenure','TotalCharges','MonthlyCharges','Female','Partner','SeniorCitizen'],axis=1).columns
feature_importance[feature_importance[0].str.contains('|'.join(keep))]
feature_importance.columns = ['feature','importance']
feature_importance

A lower feature importance value means that the increasing that variable leads to decreasing the target variable. In this case, it means including this variable leads to decreasing the churn. Phone service, a two-year contract, and online security have the lowest feature importance values. Whereas, fiber optic, a month-to-month contract, and paperless billing have the highest feature values. This means that these features are the most related to churn. 

While there is no one-size-fits-all approach to reducing churn in the telecom industry, some of the factors mentioned above can have an impact on customer retention.

For example, offering a two-year contract can help lock in customers and prevent them from churning to other providers. Providing tech support and online security can also help customers feel more secure and satisfied with their service, leading to lower churn rates and thus, more revenue for the company.

On the other hand, offering a month-to-month contract may make it easier for customers to switch providers if they find a better deal elsewhere. Additionally, paperless billing and multiple lines may not be enough to retain customers if they are not satisfied with the overall quality of the service or if they have had negative experiences with customer support.

Ultimately, reducing churn in the telecom industry requires a customer-centric approach that focuses on understanding customers' needs and preferences, providing high-quality service, and continuously improving the customer experience. This may involve offering flexible contracts, investing in network quality and infrastructure, providing excellent customer support, and offering competitive pricing and promotions that meet customers' evolving needs.

## Tactics
<a id= "tactics"></a>

In [ ]:
def show_profit_of_tactic(df,col):
    """
    Take a column and show how much including that service will increase the revenue of the company.
    """
    X = df[(df[col]== 0) & (df.Churn == 1)].drop(['customerID','Churn'],axis=1)
    y = lr.predict(X)
    dft = X
    dft['y'] = y
    return sum(dft[dft.y == 0].MonthlyCharges)

### Tactic 1

I will try modeling the revenue when phone service is included for those without phone service.

In [ ]:
show_profit_of_tactic(df,'PhoneService')

The company can gain $3,356.64 from adding phone service.

## Tactic 2

In this section, I will be modeling whether adding tech support will lead to an increase in revenue.

In [ ]:
show_profit_of_tactic(df,'TechSupport')

The company will gain $36,905 from adding tech support.